In [41]:
import os
from pathlib import Path
import sys
import _pickle as pickle
import json
import copy 
import pandas,csv
import random

In [42]:
oneData = []
twoData = []
threeData = []

for root, dirs, files in os.walk("./final/gen"):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            
            with open(fileName) as json_file:
                
                try:
                    data = json.load(json_file)
                    
                    if 'oneContext' in fileName:
                        oneData.extend(data)
                    
                    elif 'twoContext' in fileName:
                        twoData.extend(data)
                        
                    else:
                        threeData.extend(data)
                        
                    
                        
                    
                    
                    
                except Exception as e: print(e)
                    

In [43]:
bucketSize = 4

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   

In [44]:
allData = []
uniqueSet = set()

personOne = "<b>Person 1: </b>"
personTwo = "<b>Person 2: </b>"


for root, dirs, files in os.walk("./complete/clean/gen"):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            
            with open(fileName) as json_file:
                
                try:
                    data = json.load(json_file)
                                    
                    currentFileName = data['currentFileName']


                    dialogs = data['dialogs']
                    
                    

                    for dialog in dialogs:

                        convObject = {}
                        currStr = ""

                        providedConv = dialog['context']['providedConversation']
                        userResponse = dialog['context']['userResponse']
                        localIndex = dialog['context']['localIndex']
                        contextLen = dialog['context']['contextLen']
                        fileIndex = dialog['context']['fileIndex']

                        uniqueStr = 'context=' + str(contextLen) + 'index=' + str(fileIndex)

                        if(uniqueStr in uniqueSet):
                            #print(uniqueStr)
                            continue

                        uniqueSet.add(uniqueStr)
                        
                        
                        if(contextLen == 1):
                            dataSource = oneData
                        elif(contextLen == 2):
                            dataSource = twoData
                        else:
                            dataSource = threeData
                            
                        foundObj = [x for x in dataSource if x['index'] == fileIndex][0]
                        conversation = foundObj['conversation'] 
                        suggestions = foundObj['suggestions']

                        for i,sugg in enumerate(suggestions):
                            if(userResponse.strip() == sugg.strip()):
                                continue

                            suggObj = {}

                            suggObj['suggIndex'] = i
                            currStr = ""

                            leftFloat = True

                            for j in range(0, len(providedConv)):
                                if (leftFloat is True):
                                    currStr += "<div class='conv leftConv'>" + personOne + providedConv[j] + "</div>"
                                else:
                                    currStr += "<div class='conv rightConv'>" + personTwo + providedConv[j] + "</div>"

                                leftFloat = not leftFloat


                            if (leftFloat is True):
                                respStr = "<div class='response leftConv'>" + personOne + sugg + "</div>"
                            else:
                                respStr = "<div class='response rightConv'>" + personTwo + sugg + "</div>"


                            suggObj['conversation'] = currStr + "<br />" +respStr
                            suggObj['contextLen'] = dialog['context']['contextLen']
                            suggObj['fileIndex'] = dialog['context']['fileIndex']

                            allData.append(suggObj)
                                
                    
                    
                    
                    
                except Exception as e: print(e)
                    
                    
#random.shuffle(allData)

In [45]:
len(allData)


80

In [46]:
allData[0]

{'suggIndex': 0,
 'conversation': "<div class='conv leftConv'><b>Person 1: </b>I grew up with 8 older siblings, pretty crazy right? I think it has something to do with a fertility factor, but I have no idea what that is really, can you explain it?</div><div class='conv rightConv'><b>Person 2: </b>Whoa 8!!! Fertility factor has to do with how likely a person is to have children. </div><div class='conv leftConv'><b>Person 1: </b>Oh I see, I guess my mother had quite a high fertility factor then.</div><div class='conv rightConv'><b>Person 2: </b>Yes she did.  Fertility factors  is based on a lot of things. </div><div class='conv leftConv'><b>Person 1: </b>Alright, can you elaborate on what some of those things are?</div><br /><div class='response rightConv'><b>Person 2: </b>I have 2. She was a child and she was a child. She was a child and she was a child.</div>",
 'contextLen': 3,
 'fileIndex': 27}

In [47]:
divided = list(chunks(allData, bucketSize))

In [48]:
len(divided)

20

In [49]:
padConv = {
            'conversation': '<div class="conv leftConv"><b>Person 1: </b>Hi, can you tell me about iPhone?</div><br /><div class="response rightConv"><b>Person 2: </b>Exercise is useful.</div>',
             'contextLen': -2,
             'fileIndex': -1
          }

In [50]:
badConv = {
            'conversation': '<div class="conv leftConv"><b>Person 1: </b>Hi, can you tell me about iPhone?</div><br /><div class="response rightConv"><b>Person 2: </b>Exercise is useful.</div>',
             'contextLen': -1,
             'fileIndex': -1
          }

In [51]:
nLast = len(divided[-1])

while nLast < bucketSize:
    divided[-1].append(padConv)
    
    nLast += 1

In [52]:
finalData = []

In [53]:
for sample in divided:
    
    sample.append(badConv)
    random.shuffle(sample)
    
    finalObj = {}
    
    for i in range(len(sample)):
    
        conversationKey = 'conversation-vt-' + str(i+1)
        finalObj[conversationKey] = sample[i]['conversation']
        
        contextLenKey = 'contextLen-vt-' + str(i+1)
        finalObj[contextLenKey] = sample[i]['contextLen']
        
        fileIndexKey = 'fileIndex-vt-' + str(i+1)
        finalObj[fileIndexKey] = sample[i]['fileIndex']
        
    
    
    finalData.append(finalObj)
        


In [57]:
finalData[15]

{'conversation-vt-1': "<div class='conv leftConv'><b>Person 1: </b>I grew up in a big family with 6 other siblings.  What about you?</div><div class='conv rightConv'><b>Person 2: </b>a full siblings is a first degree relative</div><div class='conv leftConv'><b>Person 1: </b>It is indeed. I have 4 sisters and 3 brothers.  It was definitely a busy household growing up.</div><div class='conv rightConv'><b>Person 2: </b>i have grown up with 3 sisters and 1 brothers and my family is  also a busy household growing up</div><div class='conv leftConv'><b>Person 1: </b>Its lovely to have so many close relations though.  I wouldnt change it for the world.  I think these days though, families are getting smaller with much less children.  Would you agree?</div><br /><div class='response rightConv'><b>Person 2: </b>I have 2 brothers, so i have had a couple of kids, but i have never had one.</div>",
 'contextLen-vt-1': 3,
 'fileIndex-vt-1': 10,
 'conversation-vt-2': "<div class='conv leftConv'><b>Per

In [14]:
CSVFile = open('./suggestions_file_Phase2.csv', 'w') 
csv_writer = csv.writer(CSVFile) 

header = finalData[0].keys() 
csv_writer.writerow(header) 

for data in finalData:
    csv_writer.writerow(data.values()) 
  
CSVFile.close() 

In [124]:
#[x for x in allData if x['fileName'] == '20200212-010150_674_live.pkl.json']

In [125]:
with open('o_data_file_SIGIR.csv') as f:
    oreader = csv.DictReader(f)
    orows = list(oreader)

In [126]:
with open('data_file_SIGIR.csv') as f:
    reader = csv.DictReader(f)
    rows = list(reader) 

In [127]:
len(orows)

1052

In [128]:
len(rows)

918

In [129]:
#[x for x in orows if x['fileName'] == '20200212-010150_674_live.pkl.json']

In [130]:
#orows[0]

In [131]:
def Diff(li1, li2): 
    li_dif = [i for i in li1 if i not in li2] 
    return li_dif 

In [132]:
diff = Diff(rows, orows)

In [133]:
len(diff)

107

In [134]:
diff[0].values()

odict_values(["<div class='conv leftConv'><b>Person 1: </b>My favorite type of movie is a horror movie!  Are you familiar with any horror movies?</div><div class='conv rightConv'><b>Person 2: </b>I like Stephen King movies. Some horror I like </div><div class='conv leftConv'><b>Person 1: </b>I've always liked Dracula - I think those movies have been around for a long time!  Do you know of any of the early Dracula actors?</div><div class='conv rightConv'><b>Person 2: </b>I remember the books mostly , like Edgar Allen Poe, Bram Stoker and Mary Shelley</div><div class='conv leftConv'><b>Person 1: </b>How about zombie movies?  I think that the Walking Dead is based on a zombie movie - do you know which one?</div><br /><div class='response rightConv'><b>Person 2: </b>The Walking Dead is an ongoing black-and-white comic book series created by writer Robert Kirkman and artist Tony Moore.</div>", '20200220-193811_96_live.pkl.json', '3', '3', '10'])

In [135]:
CSVFile = open('./batch2_data_file_SIGIR.csv', 'w') 
csv_writer = csv.writer(CSVFile) 

header = diff[0].keys() 
csv_writer.writerow(header) 

for data in diff:
    csv_writer.writerow(data.values()) 
  
CSVFile.close() 